### Talk Time

In [51]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [52]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [53]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# youtube read data
sample_num = 1  # 6
time_shift = 0.6

In [54]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [55]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [56]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [57]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [58]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/1-Talk Time/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [59]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [60]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [61]:
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [62]:
#df_word_select.to_excel(f"{lang_folder.capitalize()}_200_Word.xlsx", index=False)

In [63]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [64]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:02.501,dit is de pangale v,BoaYsdPtJYA
1,00:00:04.501,00:00:05.434,unreal,BoaYsdPtJYA
2,00:00:08.518,00:00:13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,00:00:13.066,00:00:14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,00:00:14.316,00:00:15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,00:01:37.135,00:01:47.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,00:01:47.040,00:01:49.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,00:01:49.340,00:01:53.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,00:01:53.600,00:01:57.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [65]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [66]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.000,2.501,dit is de pangale v,BoaYsdPtJYA
1,4.501,5.434,unreal,BoaYsdPtJYA
2,8.518,13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,13.066,14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,14.316,15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,97.135,107.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,107.040,109.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,109.340,113.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,113.600,117.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [67]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "cElhIDdGz7M"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,cElhIDdGz7M,https://www.youtube.com/watch?v=cElhIDdGz7M&t=0s


In [68]:
df_word_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_Manuel.xlsx")
df_word_link

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik,196,197,waar ik ook zelf veel persoonlijke voorbeelden...,pruDyt__jYs,https://www.youtube.com/watch?v=pruDyt__jYs&t=...
1,je,96,97,nou dat is wel heel specifiek voor het mbo dat...,smo9IsqShlc,https://www.youtube.com/watch?v=smo9IsqShlc&t=96s
2,het,1110,1111,zwart je ziet het hier overlopen in elkaar en ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...
3,de,994,996,wat mistige beeld je fotos zal minder scherp o...,5XGbMbvvfyY,https://www.youtube.com/watch?v=5XGbMbvvfyY&t=...
4,dat,130,131,en ja ook als burgemeester hoop ik dat deze lo...,e4K8PJpTssk,https://www.youtube.com/watch?v=e4K8PJpTssk&t=...
...,...,...,...,...,...,...
195,genoeg,1307,1309,in de praktijk gewoon niet zo blijkt te zijn e...,GOkG8Yq9ANo,https://www.youtube.com/watch?v=GOkG8Yq9ANo&t=...
196,wist,1644,1645,wist dat meisje zich vier getallen te onthoude...,XRBb17aWENc,https://www.youtube.com/watch?v=XRBb17aWENc&t=...
197,oh,1924,1926,dan denk ik oh ja,xeAlsYp5XX0,https://www.youtube.com/watch?v=xeAlsYp5XX0&t=...
198,klaar,10442,10443,en heren daarmee zijn wij klaar met de,R4Q1cNbNKU0,https://www.youtube.com/watch?v=R4Q1cNbNKU0&t=...


In [69]:
df_word_link[df_word_link["search_string"].duplicated()]

,search_string,start_time,end_time,sentence,video_id,video_url


In [70]:
df_twogram_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_Manuel.xlsx")
df_twogram_link

,search_string,start_time,end_time,sentence,video_id,video_url
0,dank u,5010,5011,hamster wethouder ja dank u wel voorzitter dat...,uJhEbO42qRo,https://www.youtube.com/watch?v=uJhEbO42qRo&t=...
1,nog niet,107,109,maar kan jij even uitleggen voor de kijkers th...,5S5QX4ZK3Q0,https://www.youtube.com/watch?v=5S5QX4ZK3Q0&t=...
2,ja ja,2889,2891,zo lekker om dat te kunnen zeggen ja ja nee ik...,wehip3eUEi4,https://www.youtube.com/watch?v=wehip3eUEi4&t=...
3,niet nu,12560,12561,dat is niet nu pas begonnen daar zijn we al we...,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
4,doe maar,3919,3920,oké doe maar wat je zegt is dat ontwikkeling g...,EmxRrMXp3UY,https://www.youtube.com/watch?v=EmxRrMXp3UY&t=...
...,...,...,...,...,...,...
107,misschien nog,1774,1777,hebben bij provincies en gemeenten zal dat mis...,KFkJCia8L5M,https://www.youtube.com/watch?v=KFkJCia8L5M&t=...
108,om dit,242,244,om dit niet ongemerkt voorbij te laten gaan he...,OIb3c0hWNrI,https://www.youtube.com/watch?v=OIb3c0hWNrI&t=...
109,alles over,745,747,je er alles over maar als je zou willen,wJZCHf6RQ8c,https://www.youtube.com/watch?v=wJZCHf6RQ8c&t=...
110,echt heel,626,628,dan echt heel strak elke dag gewoon,yqD5ixgnKqI,https://www.youtube.com/watch?v=yqD5ixgnKqI&t=...


In [71]:
df_threegram_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_Manuel.xlsx")
df_threegram_link

,search_string,start_time,end_time,sentence,video_id,video_url
0,dat zal wel,369,371,maar dat zal wel goed komen ik vind jou nu in ...,HKVUi2B3AV4,https://www.youtube.com/watch?v=HKVUi2B3AV4&t=...
1,niet voor mij,4367,4369,is waren ze ontkende te nemen jullie hoeven ni...,Co4vE_sSnZM,https://www.youtube.com/watch?v=Co4vE_sSnZM&t=...
2,ja ja ja,619,621,dat is bijna onmogelijk ja ja ja precies,ULhra7fWWqE,https://www.youtube.com/watch?v=ULhra7fWWqE&t=...
3,wat was het,3265,3267,natuurlijk langs verschillende kanten bekijken...,sMaUZFkQ6QM,https://www.youtube.com/watch?v=sMaUZFkQ6QM&t=...
4,we doen het,1637,1639,het antwoord was elke keer dat dat niet nodig ...,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
5,ik weet dat,377,379,weg kon dan heb ik geen genoeg dus om nu ze we...,ftwsaGlOEdA,https://www.youtube.com/watch?v=ftwsaGlOEdA&t=...
6,dit is echt,412,414,dit is echt een rat kijk maar naar het hoofd,IIpCTf5UAMA,https://www.youtube.com/watch?v=IIpCTf5UAMA&t=...
7,dan gaan we,771,773,en dan gaan we een bordje maken,IVUi52Ejpfs,https://www.youtube.com/watch?v=IVUi52Ejpfs&t=...
8,maar ik dacht,41,44,maar ik dacht als ik de hoek van mijn bladen,fphTOzDV2ZQ,https://www.youtube.com/watch?v=fphTOzDV2ZQ&t=41s
9,was het goed,224,226,als de deur niet klemde was het goed ok,hx_YcOUrtPk,https://www.youtube.com/watch?v=hx_YcOUrtPk&t=...


In [72]:
df_sentence_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample_Selected_Manuel.xlsx")
df_sentence_link

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
1,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
2,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...,"[20, 11, 54, 63, 58, 18, 0, 159, 19, 47, 111, ..."
3,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...,"[88, 48, 0, 90, 146, 4, 2, 141, 167, 5, 8, 0, ..."
4,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...,"[11, 49, 92, 141, 113, 115, 114, 29, 30, 15, 5..."
5,je wordt ook heel wat wel dan zou je allemaal ...,431,436,waarmee je wordt ook heel wat wel dan zou je a...,5bGpXBIbiXw,https://www.youtube.com/watch?v=5bGpXBIbiXw&t=...,"[1, 135, 57, 141, 10, 47, 35, 64, 1, 148, 79, ..."
6,als ze zeg maar op je af komen en of je weet g...,615,620,gewoon heel enge als ze zeg maar op je af kome...,Fn5erRPierY,https://www.youtube.com/watch?v=Fn5erRPierY&t=...,"[25, 13, 130, 18, 17, 1, 145, 118, 8, 65, 1, 3..."
7,misschien nog even dank u zeggen aan de mensen...,226,230,misschien nog even dank u zeggen aan de mensen...,GMsJ12_DaK8,https://www.youtube.com/watch?v=GMsJ12_DaK8&t=...,"[103, 42, 111, 166, 33, 126, 32, 3, 115, 20, 1..."
8,vind voor die andere beter dan gaan we daar we...,101,106,deze toneel vind voor die andere beter dan gaa...,5pDwEvEN_dA,https://www.youtube.com/watch?v=5pDwEvEN_dA&t=...,"[194, 21, 20, 165, 181, 35, 63, 11, 70, 125, 3..."
9,een keer tegen mij zij die niet voor mij zijn ...,360,365,zei ooit een keer tegen mij zij die niet voor ...,LyPygt9GFlk,https://www.youtube.com/watch?v=LyPygt9GFlk&t=...,"[6, 164, 127, 61, 161, 20, 7, 21, 61, 16, 16, ..."


In [73]:
df_sentence_link_word_count = word_count_result(df_sentence_link, ["search_string"], set_condition=False)
df_sentence_link_word_count

,word,word_count
0,en,7
1,wel,7
2,dat,7
3,maar,7
4,we,7
...,...,...
101,zal,1
102,weer,1
103,wij,1
104,zien,1


In [74]:
set_sent_link_word = set(df_sentence_link_word_count["word"])
set_word_list = set(word_list)

In [75]:
set_word_list.difference(set_sent_link_word)

{'alleen',
 'bedankt',
 'bent',
 'binnen',
 'dag',
 'deze',
 'doet',
 'dood',
 'door',
 'eens',
 'ga',
 'gaat',
 'gedaan',
 'geef',
 'geld',
 'genoeg',
 'gezien',
 'h',
 'haar',
 'had',
 'heb',
 'hebt',
 'heeft',
 'hem',
 'hun',
 'iemand',
 'iets',
 'in',
 'jaar',
 'jij',
 'jou',
 'jouw',
 'kan',
 'klaar',
 'kom',
 'komt',
 'kon',
 'kunt',
 'laat',
 'leven',
 'm',
 'mag',
 'maken',
 'man',
 'me',
 'mee',
 'met',
 'mijn',
 'moeder',
 'moeten',
 'mr',
 'n',
 'na',
 'nee',
 'neem',
 'net',
 'niemand',
 'niets',
 'niks',
 'nooit',
 'ok',
 'omdat',
 'ons',
 'onze',
 'praten',
 's',
 'spijt',
 't',
 'toen',
 'tot',
 'twee',
 'uit',
 'uw',
 'vader',
 'vinden',
 'vrouw',
 'waar',
 'waarom',
 'wacht',
 'waren',
 'weg',
 'werd',
 'werk',
 'willen',
 'wist',
 'z',
 'zei',
 'zich',
 'zie',
 'zit',
 'zitten',
 'zo',
 'zoals',
 'zullen'}

In [76]:
# used for multi search result
#twogram_link_list = df_twogram_link["search_string"].to_list()
#threegram_link_list = df_threegram_link["search_string"].to_list()
df_result = pd.DataFrame()
for sent in df_sentence_link["search_string"]:

    # words
    sent_words = word_tokenize(sent)

    # twogram
    twogram_zip = ngrams(sent.split(), 2)
    twogram_list = [" ".join(x) for x in twogram_zip]
    #df_twogram_var = pd.DataFrame(data=twogram, columns=["twogram"])
    
    # threegram
    threegram_zip = ngrams(sent.split(), 3)
    threegram_list = [" ".join(y) for y in threegram_zip]
    #df_threegram_var = pd.DataFrame(data=threegram, columns=["threegram"])

    # begin sent repeat 
    df_sent_search_begin = df_sentence_link[df_sentence_link["search_string"] == sent]
    for repeat_num in range(3):
        df_result = pd.concat([df_result,df_sent_search_begin], axis=0)

    # word result
    for word in sent_words:        
        df_word_search_var = df_word_link[df_word_link["search_string"] == word]
        df_word_search_var.reset_index(drop=True, inplace=True)
        for i in range(len(df_word_search_var)):
            df_link_default_var = df_link_default
            try:
                word_time_diff_var = df_word_search_var.loc[i,"end_time"] - df_word_search_var.loc[i,"start_time"]
                if word_time_diff_var < 4:
                    word_time_diff_var2 = 4
                else:
                    word_time_diff_var2 = word_time_diff_var+1.0                                       
                df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
                df_result = pd.concat([df_result,df_word_search_var.iloc[[i,]]], axis=0)
                df_result = pd.concat([df_result,df_link_default_var], axis=0)
            except:
                pass

    # twogram result
    df_twogram_search_var = df_twogram_link[df_twogram_link["search_string"].isin(twogram_list)]
    df_twogram_search_var.reset_index(drop=True, inplace=True)
    for j in range(len(df_twogram_search_var)):
        df_link_default_var = df_link_default
        try:
            twogram_time_diff_var = df_twogram_search_var.loc[j,"end_time"] - df_twogram_search_var.loc[j,"start_time"]
            if twogram_time_diff_var < 4:
                twogram_time_diff_var2 = 4
            else:
                twogram_time_diff_var2 = twogram_time_diff_var+1.0 
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + twogram_time_diff_var2
            df_result = pd.concat([df_result,df_twogram_search_var.iloc[[j,]]], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass

    # threegram result
    df_threegram_search_var = df_threegram_link[df_threegram_link["search_string"].isin(threegram_list)]
    df_threegram_search_var.reset_index(drop=True, inplace=True)
    for k in range(len(df_threegram_search_var)):
        df_link_default_var = df_link_default
        try:
            threegram_time_diff_var = df_threegram_search_var.loc[k,"end_time"] - df_threegram_search_var.loc[k,"start_time"]
            if threegram_time_diff_var < 4:
                threegram_time_diff_var2 = 4
            else:
                threegram_time_diff_var2 = threegram_time_diff_var+1.0 
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + threegram_time_diff_var2
            df_result = pd.concat([df_result,df_threegram_search_var.iloc[[k,]]], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass

    # sentence added
    df_sent_search_var = df_sentence_link[df_sentence_link["search_string"] == sent]
    df_sent_search_var.reset_index(drop=True, inplace=True)
    for l in range(len(df_sent_search_var)):
        df_link_default_var = df_link_default
        try:
            sent_time_diff_var = df_sent_search_var.loc[l,"end_time"] - df_sent_search_var.loc[l,"start_time"]
            if sent_time_diff_var < 4:
                sent_time_diff_var2 = 4
            else:
                sent_time_diff_var2 = sent_time_diff_var+1.0 
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + sent_time_diff_var2
            df_result = pd.concat([df_result,df_sent_search_var.iloc[[l,]]], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass

df_result.reset_index(drop=True, inplace=True)   

In [77]:
df_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
1,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
2,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
3,en,216,217,de is een tijdje lang en beleid geweest uit vo...,z7DjsWLAdMk,https://www.youtube.com/watch?v=z7DjsWLAdMk&t=...,NaN
4,repeat,5,9,repeat_again,cElhIDdGz7M,https://www.youtube.com/watch?v=cElhIDdGz7M&t=0s,NaN
...,...,...,...,...,...,...,...
1036,repeat,5,9,repeat_again,cElhIDdGz7M,https://www.youtube.com/watch?v=cElhIDdGz7M&t=0s,NaN
1037,kunnen we nog,662,664,die kunnen we nog gebruiken straks voor het ge...,W8sUTunwiVQ,https://www.youtube.com/watch?v=W8sUTunwiVQ&t=...,NaN
1038,repeat,5,9,repeat_again,cElhIDdGz7M,https://www.youtube.com/watch?v=cElhIDdGz7M&t=0s,NaN
1039,als mensen op tijd komen kunnen we nog veel me...,52,55,want als mensen op tijd komen kunnen we nog ve...,U3oEpccNgR0,https://www.youtube.com/watch?v=U3oEpccNgR0&t=52s,"[25, 115, 17, 123, 118, 84, 11, 42, 113, 79, 2..."


In [78]:
df_result[df_result["search_string"] == "repeat"]["end_time"].max()

12

In [79]:
sample_num = df_result[df_result["search_string"] == "repeat"]["end_time"].count()
sample_num

492

In [80]:
((df_result[df_result["search_string"] == "repeat"]["end_time"].sum()*2)-sample_num)/60

140.63333333333333

In [81]:
df_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time1.xlsx", index=False)

In [ ]:
part_sample_num = 116
sample_num_start = 0
sample_num_end = part_sample_num
for i in range(12):
    df_var = df_result.iloc[sample_num_start:sample_num_end,]
    
    id_list = []
    start_list = []
    end_list = []
    for id, start, end in zip(df_var["video_id"].to_list(),df_var["start_time"].to_list(),df_var["end_time"].to_list()):
        id_list.append(str(id))
        start_list.append(str(start))
        end_list.append(str(end))

    id_join = ",".join(id_list)
    start_join = ",".join(start_list)
    end_join = ",".join(end_list)

    df_result_for_embedded = pd.DataFrame(data=[[id_join,start_join,end_join]], columns=["id","start_time","end_time"])
    df_result_for_embedded.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time1_Join_Sample{i+1}.xlsx", index=False)
    #df_var.to_excel(f"Turkish_200_Word_Talk_Time1_Sample{i+1}.xlsx", index=False)
    sample_num_start += part_sample_num
    sample_num_end += part_sample_num

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Word_Talk_Time*.xlsx")
output_file

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

In [ ]:
# used for one search result
df_result = pd.DataFrame()
for sent in df_sentence_link["search_string"]:
    sent_words = word_tokenize(sent)
    for word in sent_words:
        df_link_default_var = df_link_default
        df_var = df_word_link[df_word_link["search_string"] == word]
        df_var.reset_index(drop=True, inplace=True)
        try:
            var_time_diff = (df_var.loc[0,"end_time"] - df_var.loc[0,"start_time"])
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + var_time_diff+1.0
            df_result = pd.concat([df_result,df_var], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass
df_result.reset_index(drop=True, inplace=True)
df_result

In [87]:
df_result_select = df_result.head(120)
df_result_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
1,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
2,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
3,en,216,217,de is een tijdje lang en beleid geweest uit vo...,z7DjsWLAdMk,https://www.youtube.com/watch?v=z7DjsWLAdMk&t=...,NaN
4,repeat,5,9,repeat_again,cElhIDdGz7M,https://www.youtube.com/watch?v=cElhIDdGz7M&t=0s,NaN
...,...,...,...,...,...,...,...
115,repeat,5,9,repeat_again,cElhIDdGz7M,https://www.youtube.com/watch?v=cElhIDdGz7M&t=0s,NaN
116,wel,5155,5157,dank u wel,9uOFhXkzEQQ,https://www.youtube.com/watch?v=9uOFhXkzEQQ&t=...,NaN
117,repeat,5,9,repeat_again,cElhIDdGz7M,https://www.youtube.com/watch?v=cElhIDdGz7M&t=0s,NaN
118,ja ja,2889,2891,zo lekker om dat te kunnen zeggen ja ja nee ik...,wehip3eUEi4,https://www.youtube.com/watch?v=wehip3eUEi4&t=...,NaN


In [88]:
df_result_select.to_excel("Dutch_200_Word_Talk_Time1_Test.xlsx", index=False)

In [89]:
id_list = []
start_list = []
end_list = []
for id, start, end in zip(df_result_select["video_id"].to_list(),df_result_select["start_time"].to_list(),df_result_select["end_time"].to_list()):
    id_list.append(str(id))
    start_list.append(str(start))
    end_list.append(str(end))

id_join = ",".join(id_list)
start_join = ",".join(start_list)
end_join = ",".join(end_list)

df_result_for_embedded = pd.DataFrame(data=[[id_join,start_join,end_join]], columns=["id","start_time","end_time"])
df_result_for_embedded

,id,start_time,end_time
0,"V1RX2oPECHY,V1RX2oPECHY,V1RX2oPECHY,z7DjsWLAdM...","207,207,207,216,5,994,5,3431,5,1082,5,216,5,19...","213,213,213,217,9,996,9,3433,9,1083,9,217,9,19..."


In [90]:
df_result_for_embedded.to_excel("Dutch_200_Word_Talk_Time1_Join_Test.xlsx", index=False)